In [5]:
# Impor the necessary libraries

import pandas as pd 
import numpy as np
import datetime

In [6]:
def calculate_indicators(df):
    df = df.copy()
    
    df['sma_10'] = df['close'].rolling(window=10).mean()

    # Entry trigger: Price crosses above SMA-10 from below
    df['sma_cross'] = (df['close'].shift(1) <= df['sma_10'].shift(1)) & (df['close'] > df['sma_10'])

    return df

In [7]:
def strategy_implementation(df, tp, sl):
    df = calculate_indicators(df)
    trades = []
    active_trades = []

    for i in range(10, len(df)):
        row = df.iloc[i]

        # Exit logic for active LONG trades
        exited_trades = []
        for trade in active_trades:
            exit_price = None
            exit_reason = None

            if row['low'] <= trade['sl']:
                exit_price = trade['sl']
                exit_reason = 'SL hit'
                pnl = trade['sl'] - trade['entry_price']

            elif row['high'] >= trade['tp']:
                exit_price = trade['tp']
                exit_reason = 'TP hit'
                pnl = trade['tp'] - trade['entry_price']

            elif row['close'] < row['sma_10']:  # Exit if price closes below SMA-10
                exit_price = row['close']
                exit_reason = 'SMA-10 breakdown'
                pnl = row['close'] - trade['entry_price']

            if exit_price is not None:
                trade.update({
                    'exit_time': row['timestamp'],
                    'exit_reason': exit_reason,
                    'exit_price': exit_price,
                    'pnl': pnl
                })
                trades.append(trade)
                exited_trades.append(trade)

        # Remove exited trades from active_trades
        active_trades = [t for t in active_trades if t not in exited_trades]

        # Entry logic: Price crosses above SMA-10 (already in df['sma_cross'])
        if row['sma_cross']:
            entry_price = row['close']
            active_trades.append({
                'entry_time': row['timestamp'],
                'position': 'long',
                'entry_price': entry_price,
                'sl': entry_price * (1 - sl),  # SL below entry
                'tp': entry_price * (1 + tp)   # TP above entry
            })

    df_trades = pd.DataFrame(trades)

    if df_trades.empty:
        return df_trades, {
            'total_trades': 0, 'win_rate': 0, 'avg_pnl': 0,
            'risk_reward_ratio': np.nan, 'max_drawdown': 0, 'avg_trade_per_day': 0
        }

    df_trades['wins'] = (df_trades['pnl'] > 0).astype(int)
    df_trades['cumulative_pnl'] = df_trades['pnl'].cumsum()
    initial_capital = df_trades['entry_price'].iloc[0]
    df_trades['equity_curve'] = df_trades['cumulative_pnl'] + initial_capital
    max_drawdown = (df_trades['equity_curve'].cummax() - df_trades['equity_curve']).max()

    duration_days = (df['timestamp'].max() - df['timestamp'].min()).days + 1
    metrics = {
        'total_trades': len(df_trades),
        'win_rate': df_trades['wins'].mean(),
        'avg_pnl': df_trades['pnl'].mean(),
        'risk_reward_ratio': (
            df_trades[df_trades['pnl'] > 0]['pnl'].mean() /
            abs(df_trades[df_trades['pnl'] < 0]['pnl'].mean())
            if (df_trades['pnl'] < 0).any() else np.nan
        ),
        'max_drawdown': max_drawdown,
        'avg_trade_per_day': len(df_trades) / duration_days
    }

    return df_trades, metrics


In [ ]:
if __name__ == "__main__":

    Symbol = "NIFTY"

    market = "NIFTY_50"

    df = pd.read_csv("NIFTY_2008_2020.csv")

    # Make column names lowercase
    df.columns = df.columns.str.lower()
    
    # Select required columns           
    df = df[["date" ,"time", "open", "high", "low", "close"]]

    # Combine Date + Time and convert to datetime//
    df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"])
    # df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"])
    # df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"])

##   df = df.drop(columns=["time"])
  ##    df["timestamp"] = pd.to_datetime(df["time"])
 ###df = df[["time", "open", "high", "low", "close"]]
    ##for btc 
   ###for aapl df = df[["local time", "open", "high", "low", "close"]]
  ### for aapl df["timestamp"] = pd.to_datetime(df["local time"])
   ## for appl df = df.drop(columns=["local time"]) 

    # Drop old Date and Time columns if not needed
    df = df.drop(columns=["date", "time"])

    # Reorder columns
    df = df[["timestamp", "open", "high", "low", "close"]]

    tp_sl_variants = [(0.075, 0.03), (0.06, 0.03), (0.045, 0.03), (0.03, 0.03), (0.075, 0.04), (0.075, 0.05)]

    for tp, sl in tp_sl_variants:
        print(f"\nTesting TP: {tp}, SL: {sl}")
        df2, metrics = strategy_implementation(df, tp=tp, sl=sl)

        # Convert metrics dict to a DataFrame for easy export
        metrics_df = pd.DataFrame(list(metrics.items()), columns=['Metric', 'Value'])

        # Save both to one Excel file with two sheets
        output_file = f'strategy_backtest_results_2{Symbol}_{market}_SL{sl}_TP{tp}.xlsx'

        with pd.ExcelWriter(output_file) as writer:
            df2.to_excel(writer, sheet_name='Trades', index=False)
            metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

            print(f"Saved trades and metrics to {output_file}")




Testing TP: 0.075, SL: 0.03
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.03_TP0.075.xlsx

Testing TP: 0.06, SL: 0.03
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.03_TP0.06.xlsx

Testing TP: 0.045, SL: 0.03
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.03_TP0.045.xlsx

Testing TP: 0.03, SL: 0.03
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.03_TP0.03.xlsx

Testing TP: 0.075, SL: 0.04
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.04_TP0.075.xlsx

Testing TP: 0.075, SL: 0.05
Saved trades and metrics to strategy_backtest_results_2NIFTY_NIFTY_50_SL0.05_TP0.075.xlsx
